In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.7.20

## 부분함수 만들기

## 커링 처리하기

In [2]:
fun add(a: Int, b:Int)  =  a + b                           // 덧셈 계산함수 
fun add1(a: Int) : (Int) -> Int = { b -> a + b }           // 내부 함수인 람다표현식을 반환 
fun outer(a:Int) : (Int) -> Int {                          // 함수로 부분함수 정의          
    fun inner(b:Int) : Int = a + b                         // 내부 함수 정의 
    return ::inner                                         // 내부 함수 반환 
} 

val add2 = add1(1)                                         // 첫번째 함수 실행 : 내부함수 반환
println ( add2(2))                                         // 두번째 실행 : 내부함수 실행            

val add3 = outer(1)                                        // 첫번째 함수 실행 : 내부함수 반환
println (add3(2))                                          // 두번째 실행 : 내부함수 실행

object Add {                                               // object 선언으로 함수 처리 선언 
    fun add(a: Int): Function1<Int, Int> {                 // 내부  메소드가 함수를 반환 
        return object: Function1<Int, Int> {               // object 표현식으로 실행 객체 반환 
                override fun invoke(b: Int): Int = a + b   // 내부에 invoke 메소드 작성 
            }
    }
}

println(add(100,200))                                       // 기본 함수 실행                              
val add4 = Add.add(100)                                     // object 선언 내의 메소드 실행 
println(add4(200))                                          // 내부 반환 object 표현식 실행

3
3
300
300


## 누적계산기

In [3]:
fun a(n: Int): (d: Int) -> Int {                         // 부분 함수를 정의 
    var accumulator = n                                  // 지역변수로 누적값 관리
    return { x -> accumulator += x; accumulator }        // 내부 함수 반환 : 클로저 발생 
}

val a100 = a(100)                                        // 외부 함수 실행 
println(a100(5) )                                        // 내부 함수 실행
println(a100(10))                                        // 외부 함수와 내부함수 연속 실행
println(a(100)(5) )

object A {                                               // object 표현식 작성 
    var accumulator :Int = 0                             // 누적 속성 정의 
    fun b(n : Int) : (Int) -> Int {                      // 메소드 정의 
        accumulator = n
        return object: (Int) -> Int {                    // object 표현식으로 반환 
                  override fun invoke(n: Int): Int {
                     accumulator += n
                     return accumulator;
            }
         }
    }
}

val A200 = A.b(100)
println(A200(5))                                          // 내부 함수 실행 
println(A200(10))                                         // 내부 함수 실행
println(A.b(100)(5))                                      // 외부 함수 및 내부 함수 다시 실행 

105
115
105
105
115
105


## 확장함수로 부분함수 처리하기

In [4]:
fun ((Int, Int) -> Int).partial1(x:Int) :               // 함수 자료형(인터페이스)  부분함수 추가 
                                (Int) ->Int {           // 함수를 반환 
    return {y:Int -> this(x,y)}                         // 람다표현식으로 내부 함수 정의 
}

val add = { x:Int,y:Int -> x+y}                         // 함수 정의 
var p = add.partial1(100)                               // 함수를 사용해서 부분 확장함수 실행
println(p(200))                                         // 부분함수에서 반환된 내부 함수 처리 

val mul = { x:Int,y:Int -> x*y}                         // 함수 정의      
var p1 = mul.partial1(100)                              // 부분 확장함수 실행
println(p1(200))                                        // 내부함수 실행 

fun Function2<Int,Int,Int>.partial2(x:Int) :             // 함수 자료형(인터페이스)에 부분함수 추가
                                (Int) ->Int {
    var accumulator :Int = 0
    return {y:Int -> accumulator += y;                   //람다표현식으로 반환
                     this(x,accumulator)}
}

var p2 = add.partial2(100)                               // 부분함수를 실행 
println(p2(200))
println(p2(200))

var p3 = mul.partial2(100)
println(p3(200))
println(p3(200))

300
20000
300
500
20000
40000


## 제너릭한 부분함수 처리

In [5]:
fun <P1, P2, R> ((P1, P2) -> R).curry(x: P1) : (P2) -> R {
    return { y: P2 -> this(x,y) }
}

val add = { x:Int,y:Int -> x+y}                         // 함수 정의
var p = add.curry(100)                                  // 함수를 사용해서 부분 확장함수 실행
println(p(200))                                         // 부분함수에서 반환된 내부 함수 처리


val mul = { x:Double, y:Double -> x*y}                  // 함수 정의      
var p1 = mul.curry(100.0)                               // 부분 확장함수 실행
println(p1(200.0))                                      // 내부함수 실행 

300
20000.0
